# creating static sites from notebooks and dataframes in notebooks

this is a short notebook that shows an end to end of making a static
site from notebooks, which works with markdown or html content.
by going end to end we can see the arc from indexing the site contents
to working with the html and eventually writing to disc.
this is AN implementation, not the THE implementation. it is meant for discussion with reduced complexities.

the power in the dataframe forward is replacing object orientied programming
techniques and control flow with fluent programming interfaces that operate rows and columns of data.
this approach to programming transforms traditional static site generated from OOP TO FP.

In [255]:
    from tonyfast.tonyfast.xxiv.schema_frame import tag
    import nbconvert, nbformat, bs4, shlex

    async def sh(cmd, **kwargs):
        kwargs.setdefault("stdout", asyncio.subprocess.PIPE) 
        kwargs.setdefault("stderr", asyncio.subprocess.PIPE) 
        return list(map(bytes.decode, await (await asyncio.subprocess.create_subprocess_exec(*shlex.split(cmd), **kwargs)).communicate()))


## create the index of input files

the substrate for all static site data frame work is creating an index of all the content files.
typically, a static site is generated from a git repo, and it is possible request things like
updated times and authors from the revision history.

In [256]:
index = Index(Path("~/tonyfast/tonyfast/xxiv/").expanduser().glob("*.ipynb")).rename("path")

In [257]:
%%
load in all of the file contents

    df = Series(
        await gather(*index.map(compose_left(anyio.Path, anyio.Path.read_text))), index
    ).apply(json.loads).rename("data").to_frame()

<details><summary>ensure some types in the notebook formats</summary>

    for nb in df.data:
        for cell in nb["cells"]:
            cell["source"] = "".join(cell["source"])
            for output in cell.get("outputs", ""):
                if "data" in output:
                    for k, v in output["data"].items():
                        if k == "text/markdown":
                            output["data"][k] = "".join(v)
</details>



now that we have out data structured we can perform simple operations like creating a target for the content in a static site context.

    df = df.index.to_series().apply(lambda x: Path(x.with_suffix(x.suffix + ".html").name)).to_frame("target").combine_first(df)
    df = df.head(20)

or a more complicated scenario where we extract that time the  content was updated from the git history.

    df = Series(await gather(*(df.index.to_series().apply(
        lambda x: sh("""git log --oneline -n1  --pretty="format:%H %ct" -- """ + x.name + "", cwd=x.parent)
    ))), df.index).apply(first).str.split(expand=True).rename(columns={0: "hash", 1: "updated_at"}).combine_first(df)

to recap, from our index we read in the files contents and extract file level metadata from teh dataframe. these actions represent some of the ways we can work with documentation as structured data.

{{df.T._repr_html_()}}

path,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-01-29-ravelry.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-01-23-ipython.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled10.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled7.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-03-15-screen-tests.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-04-04-nbconvert-html-screen-reader.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-07-03-axes.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-04-03-markdown-lists-to-python.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-05-11-stream.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-02-28-histograms-sucks.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-03-01-a11y-list-string.ipynb.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-04-17-windchime.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled12.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled5.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled1.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-11-01-stream.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-12-07-reuse.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-06-10-illusion.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-08-08-toggle.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-02-03-github-graphql.ipynb
data,"{'cells': [{'cell_type': 'markdown', 'id': '64...","{'cells': [{'attachments': {}, 'cell_type': 'm...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'attachments': {}, 'cell_type': 'm...","{'cells': [{'cell_type': 'markdown', 'id': 'bd...","{'cells': [{'cell_type': 'markdown', 'id': 'a0...","{'cells': [{'cell_type': 'markdown', 'id': 'fc...","{'cells': [{'cell_type': 'markdown', 'id': 'd2...","{'cells': [{'attachments': {}, 'cell_type': 'm...",{'cells': [{'attachments': {'931392ee-8c73-455...,{'cells': [{'attachments': {'5db474ff-aee9-4d3...,"{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'markdown', 'id': '0d...","{'cells': [{'cell_type': 'markdown', 'id': 'b0...","{'cells': [{'cell_type': 'markdown', 'id': 'b7...","{'cells': [{'cell_type': 'markdown', 'id': '2f...","{'cells': [{'cell_type': 'markdown', 'id': 'f4...","{'cells': [{'cell_type': 'markdown', 'id': '3c..."
hash,35a5eb8635cd8b3f693d12c8dd144ee131f6bdda,40d3942fc512693b44e43940616b1833b9312fdb,None,None,fb49fa3d718a268c739a4deeff3740960b8646f2,69699fd006cb56cc5312bb21f10205851f833f9c,8f50e162948108a1c002104afdeb19f6b7f6debf,9885704a54f17616dc3d7fb40a6ebb36290bfb40,a5de2fb8d737c9f56721044e0c107feb24a37d13,628c352d42745c4afcb2cef8847fa27b80db3698,f4b6cce6957117943750236e1060cb51260a6825,3f327a9a720b0e2042126b63f3494c46e341f2e9,None,None,None,fad9916af968bb1bb714901a1fcd9851fba7a37e,2a606c28836791f630be936bff8552ff28c0ed04,042a0aa50b96b193855e84fc174491003af4b9d4,5c47e49f984186f0ad2f0b742b1f8c5d48d3677f,84c8eea29003e6bff11249666f8210dd425581a4
target,2025-01-29-ravelry.ipynb.html,2025-01-23-ipython.ipynb.html,Untitled10.ipynb.html,Untitled7.ipynb.html,2024-03-15-screen-tests.ipynb.html,2024-04-04-nbconvert-html-screen-reader.ipynb....,2024-07-03-axes.ipynb.html,2024-04-03-markdown-lists-to-python.ipynb.html,2024-05-11-stream.ipynb.html,2024-02-28-histograms-sucks.ipynb.html,2024-03-01-a11y-list-string.ipynb.ipynb.html,2024-04-17-windchime.ipynb.html,Untitled12.ipynb.html,Untitled5.ipynb.html,Untitled1.ipynb.html,2024-11-01-stream.ipynb.html,2024-12-07-reuse.ipynb.html,2025-06-10-illusion.ipynb.html,2024-08-08-toggle.ipynb.html,2025-02-03-github-graphql.ipynb.html
updated_at,1738211637,1737662379,None,None,1710544502,1712344757,1720045170,1712205886,1715541924,1709225483,1709329306,1713393508,None,None,None,1730529025,1733607019,1749610855,1723175809,1738697411


In [258]:
%%
## rendering html

the approach using the notebook format as a specification for loading files as structure data.
from the `nbformat` we can produce files and archives in many formats. the singular target of html outputs 
in static sites makes it hard to generate other formats. the <var>exporter</var> transforms notebook formats into full html pages,
including rendering markdown and 

    exporter = nbconvert.get_exporter("html")(embed_images=True)
    df = df.data.apply(compose_left(nbformat.from_dict, exporter.from_notebook_node, first)).to_frame("html").combine_first(df)
    df = df["html"].apply(bs4.BeautifulSoup, features="lxml").to_frame("bs4").combine_first(df)


now our expanded dataframe includes the content as html, and a beautiful soup object that provides post processing abilities.
       
{{df.T._repr_html_()}}

/Users/tonyfast/.pixi/envs/default/share/jupyter/nbconvert/templates/base/display_priority.j2:32: UserWarning: Your element with mimetype(s) dict_keys(['text/x-python']) is not able to be represented.
  {%- elif type == 'text/vnd.mermaid' -%}


path,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-01-29-ravelry.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-01-23-ipython.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled10.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled7.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-03-15-screen-tests.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-04-04-nbconvert-html-screen-reader.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-07-03-axes.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-04-03-markdown-lists-to-python.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-05-11-stream.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-02-28-histograms-sucks.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-03-01-a11y-list-string.ipynb.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-04-17-windchime.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled12.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled5.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/Untitled1.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-11-01-stream.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-12-07-reuse.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-06-10-illusion.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2024-08-08-toggle.ipynb,/Users/tonyfast/tonyfast/tonyfast/xxiv/2025-02-03-github-graphql.ipynb
bs4,"[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met..."
data,"{'cells': [{'cell_type': 'markdown', 'id': '64...","{'cells': [{'attachments': {}, 'cell_type': 'm...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'attachments': {}, 'cell_type': 'm...","{'cells': [{'cell_type': 'markdown', 'id': 'bd...","{'cells': [{'cell_type': 'markdown', 'id': 'a0...","{'cells': [{'cell_type': 'markdown', 'id': 'fc...","{'cells': [{'cell_type': 'markdown', 'id': 'd2...","{'cells': [{'attachments': {}, 'cell_type': 'm...",{'cells': [{'attachments': {'931392ee-8c73-455...,{'cells': [{'attachments': {'5db474ff-aee9-4d3...,"{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'markdown', 'id': '0d...","{'cells': [{'cell_type': 'markdown', 'id': 'b0...","{'cells': [{'cell_type': 'markdown', 'id': 'b7...","{'cells': [{'cell_type': 'markdown', 'id': '2f...","{'cells': [{'cell_type': 'markdown', 'id': 'f4...","{'cells': [{'cell_type': 'markdown', 'id': '3c..."
hash,35a5eb8635cd8b3f693d12c8dd144ee131f6bdda,40d3942fc512693b44e43940616b1833b9312fdb,None,None,fb49fa3d718a268c739a4deeff3740960b8646f2,69699fd006cb56cc5312bb21f10205851f833f9c,8f50e162948108a1c002104afdeb19f6b7f6debf,9885704a54f17616dc3d7fb40a6ebb36290bfb40,a5de2fb8d737c9f56721044e0c107feb24a37d13,628c352d42745c4afcb2cef8847fa27b80db3698,f4b6cce6957117943750236e1060cb51260a6825,3f327a9a720b0e2042126b63f3494c46e341f2e9,None,None,None,fad9916af968bb1bb714901a1fcd9851fba7a37e,2a606c28836791f630be936bff8552ff28c0ed04,042a0aa50b96b193855e84fc174491003af4b9d4,5

In [259]:
%%
## indexes as joins

commonly, a static site generator will aggregate blog posts using a seperate template,
the dataframe approach doesn't require a switch in interfaces. in dataframe parlance,
we performing joins on dataframe elements

we extract the titles from the rendered html content.

    df = df.bs4.apply(
        bs4.Tag.select_one, args=("h1,h2,h3,h4,h5,h6",)
    ).dropna().apply(bs4.Tag.get_text).to_frame("title").combine_first(df).combine_first(
        df.index.to_series().apply(compose_left(operator.attrgetter("stem"))).to_frame("title")
    )

then we can take all of the titles and render them as html elements.

    index = tag.section(
        tag.h1("blog posts"),
        df.title.html.tag("a", href=df.target).html.tag("li").html.group("ol")
    )

<details><summary>sample <var>index</var></summary>
{{index}}
</details>

now we can do a spot check of our <var>index</var> elements representation
before we aggregate it into an entire webpage.

    indexes = Series([nbformat.v4.new_notebook(cells=[
        nbformat.v4.new_markdown_cell(str(index))
    ])], Index([Path("index.html")], name="target")).to_frame("data")
    indexes = indexes.data.apply(compose_left(
        nbformat.from_dict, exporter.from_notebook_node, first
    )).to_frame("html").combine_first(indexes)

we can use a similar technique to cast the dataframe as an `atom.rss` or `feed.xml` file

indexes as joins 
 commonly, a static site generator will aggregate blog posts using a seperate template,
the dataframe approach doesn't require a switch in interfaces. in dataframe parlance,
we performing joins on dataframe elements 
 we extract the titles from the rendered html content. 
 df = df.bs4.apply(
 bs4.Tag.select_one, args=("h1,h2,h3,h4,h5,h6",)
).dropna().apply(bs4.Tag.get_text).to_frame("title").combine_first(df).combine_first(
 df.index.to_series().apply(compose_left(operator.attrgetter("stem"))).to_frame("title")
)
 
 then we can take all of the titles and render them as html elements. 
 index = tag.section(
 tag.h1("blog posts"),
 df.title.html.tag("a", href=df.target).html.tag("li").html.group("ol")
)
 
 sample index 
 blog posts histograms suck¶ using microdata and semantic html to represent python objects¶ revisiting nbviewer/nbconvert screen reader experiences¶ turning markdown lists to python objects¶ screen reader improvements to nbconvert html exporter¶ wind chime table¶ stream drawing words with video elements¶ making figure axes from grouped table headers¶ toggle between a table, histogram, and scatter plot¶ november 1, 2024 stream¶ a reusable technique for markdown attachemnts¶ how to hack an ipython kernel and shell¶ extracting accessibility related patterns from ravelry¶ gathering metadata on assistive technology¶ video circles¶ tewting¶ Untitled10 Untitled12 Untitled5 Untitled7 
 
 now we can do a spot check of our index elements representation
before we aggregate it into an entire webpage. 
 indexes = Series([nbformat.v4.new_notebook(cells=[
 nbformat.v4.new_markdown_cell(str(index))
])], Index([Path("index.html")], name="target")).to_frame("data")
indexes = indexes.data.apply(compose_left(
 nbformat.from_dict, exporter.from_notebook_node, first
)).to_frame("html").combine_first(indexes)
 
 we can use a similar technique to cast the dataframe as an atom.rss or feed.xml file

In [260]:
%%
<details open><summary><h3>pagination as groupby</h3></summary>

commonly, static site generators will have pagination indexes that limit the percievable items on a page.
this is a natural groupby action with a dataframe.

    grouped_indexes = df.groupby(RangeIndex(len(df))//5).apply(
        lambda df: Series([tag.section(
            tag.h1("blog posts"),
            df.title.html.tag("a", href=df.target).html.tag("li").html.group("ol")
        )], [F"index{df.name and str(df.name) or ""}.html"])
    )
    
    grouped_indexes = grouped_indexes.apply(
        lambda index: nbformat.v4.new_notebook(cells=[
            nbformat.v4.new_markdown_cell(str(index))
        ])
    ).to_frame("data").reset_index(0, drop=True).rename_axis(index="target")
    
    grouped_indexes = grouped_indexes.data.apply(compose_left(
        nbformat.from_dict, exporter.from_notebook_node, first
    )).to_frame("html").combine_first(grouped_indexes)

{{grouped_indexes.T._repr_html_()}}

</details>

target,index.html,index1.html,index2.html,index3.html
data,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata..."
html,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me..."


In [261]:
%%
## writing the files

writing the files flips our index from the source content to the target content

    targets = df.reset_index().set_index("target").combine_first(indexes)

{{targets.T._repr_html_()}}

the target frame provides all the information to write our static site to disc.

    for target, row in targets.iterrows():
        target = "site" / target
        target.parent.mkdir(parents=True, exist_ok=True)
        target.write_text(str(row.html))

we wrote <data>{{len(targets)}}</data> files to disc, and the contents are show in the list below


{{Series(Path("site").rglob("*.html")).to_frame("contents").T._repr_html_()}}

target,2024-02-28-histograms-sucks.ipynb.html,2024-03-01-a11y-list-string.ipynb.ipynb.html,2024-03-15-screen-tests.ipynb.html,2024-04-03-markdown-lists-to-python.ipynb.html,2024-04-04-nbconvert-html-screen-reader.ipynb.html,2024-04-17-windchime.ipynb.html,2024-05-11-stream.ipynb.html,2024-07-03-axes.ipynb.html,2024-08-08-toggle.ipynb.html,2024-11-01-stream.ipynb.html,...,2025-01-23-ipython.ipynb.html,2025-01-29-ravelry.ipynb.html,2025-02-03-github-graphql.ipynb.html,2025-06-10-illusion.ipynb.html,Untitled1.ipynb.html,Untitled10.ipynb.html,Untitled12.ipynb.html,Untitled5.ipynb.html,Untitled7.ipynb.html,index.html
bs4,"[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...",...,"[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...","[html, [\n, [<meta charset=""utf-8""/>, \n, <met...",NaN
data,"{'cells': [{'attachments': {}, 'cell_type': 'm...",{'cells': [{'attachments': {'931392ee-8c73-455...,"{'cells': [{'attachments': {}, 'cell_type': 'm...","{'cells': [{'cell_type': 'markdown', 'id': 'fc...","{'cells': [{'cell_type': 'markdown', 'id': 'bd...",{'cells': [{'attachments': {'5db474ff-aee9-4d3...,"{'cells': [{'cell_type': 'markdown', 'id': 'd2...","{'cells': [{'cell_type': 'markdown', 'id': 'a0...","{'cells': [{'cell_type': 'markdown', 'id': 'f4...","{'cells': [{'cell_type': 'markdown', 'id': 'b0...",...,"{'cells': [{'attachments': {}, 'cell_type': 'm...","{'cells': [{'cell_type': 'markdown', 'id': '64...","{'cells': [{'cell_type': 'markdown', 'id': '3c...","{'cells': [{'cell_type': 'markdown', 'id': '2f...","{'cells': [{'cell_type': 'markdown', 'id': '0d...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'code', 'execution_co...","{'cells': [{'cell_type': 'code', 'execution_co...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata..."
hash,628c352d42745c4afcb2cef8847fa27b80db3698,f4b6cce6957117943750236e1060cb51260a6825,fb49fa3d718a268c739a4deeff3740960b8646f2,9885704a54f17616dc3d7fb40a6ebb36290bfb40,69699fd006cb56cc5312bb21f10205851f833f9c,3f327a9a720b0e2042126b63f3494c46e341f2e9,a5de2fb8d737c9f56721044e0c107feb24a37d13,8f50e162948108a1c002104afdeb19f6b7f6debf,5c47e49f984186f0ad2f0b742b1f8c5d48d3677f,fad9916af968bb1bb714901a1fcd9851fba7a37e,...,40d3942fc512693b44e43940616b1833b9312fdb,35a5eb8635cd8b3f693d12c8dd144ee131f6bdda,84c8eea29003e6bff11249666f8210dd425581a4,042a0aa50b96b193855e84fc174491003af4b9d4,None,None,None,None,None,NaN
html,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...",...,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html lang=""en"">\n<head><me...","<!DOCTYPE html>\n\n<html la

## conclusion 

dataframes for documentation have natural interactive affordances that improve the flow and interaction while modifying static site content. the dataframe provides a consistent API across all considerations of the site from the high-level macroscopic position of the site to the canonical pages of document, blog posts, and other media all the way down to the nitty gritty units of content.